## Tweet Topic Modeling using Latent Dirichlet Allocation (LDA)

We begin by importing dependencies.

In [2]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys
import os
import jsonpickle
import json
import re

Here we create a tweepy API object that we use to query for data from Twitter. 

In [3]:
consumer_key="9GPJWDdk99uK6BtQjsI2B1gZU"
consumer_secret="YzuuPHuilr60BRSLLlD2gyhqMuu79us84wvBbViKfm6wCmd14y"
access_token="1194296389074735105-l3oRi4yLVY9wp6iAxjLF5MGZu7IWGN"
access_token_secret="bWW2u4rXTEYqJIvi9jnteetEotfg7RwQvTriKo3EdIaR9"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

We now pull the last 100 tweets from CNN

In [40]:
tweetCount = 0;

with open('DirichletTweets.json','w') as f:
    for tweet in tweepy.Cursor(auth_api.user_timeline,id='cnn').items():
        tweety = jsonpickle.encode(tweet._json, unpicklable=False)
        f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
        tweetCount += 1
        
        if tweetCount == 100:
            break
        
print("Downloaded {0} tweets".format(tweetCount))

Downloaded 100 tweets


Now we create a list of the tweets, with each tweet being a python dictionary from which we proceed to extract
the actual content of the tweet which is stored in the "text" key of the dictionary.

In [45]:
tweets = []
for line in open('DirichletTweets.json', 'r'):
    tweets.append(json.loads(line)["text"])
#print(tweets[:10])

type(tweets[2])
tweets[9]

len(tweets)

100

We now remove punctuations and URLs.

In [47]:
#my_str = "Hello!!!, he said ---and went."
# To take input from the user
# my_str = input("Enter a string: ")

# remove punctuation from the string

def punctRemover(sentence):
    # list the different punctuations
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in sentence:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [50]:
def urlRemover(sentence):
    pattern = r"http\S+"
    return re.sub(pattern,"",sentence)

new_tweets = []
for tweet in tweets:
    new_tweets.append(urlRemover(punctRemover(tweet)))

print(new_tweets)

['Police found a fully grown 5footlong American alligator living in the basement of an Ohio home The homeowner sa… ', 'Coronavirus symptoms include a runny nose cough sore throat headache and a fever that can last for a couple of d… ', 'There is no evidence that garlic water vitamin C essential oils colloidal silver or steroids will protect yo… ', '11yearold Star Wars fan Isabella Tadlock was born with a nub on the end of her left arm and no fingers on her r… ', 'Lettuce grown in space is not only tasty and safe for astronauts to eat  its also free of diseasecausing micro… ', 'Heres how Danone yogurt went from being made in a garage to taking over the world  ', 'A quiet revolution is reshaping the agricultural world with farmers like Danie Slabbert saying that if we want to… ', 'On April 29 an asteroid estimated to be between 11 and 25 miles wide will fly by Earth But its not expected to… ', 'Meet Rollerman Wearing a cyborgstyle suit made with carbon fiber and roller skate wheels he ca

In [62]:
import pandas as pd

df = pd.DataFrame(new_tweets, columns=['Tweets'])
df.head(20)


,Tweets
0,Police found a fully grown 5footlong American ...
1,Coronavirus symptoms include a runny nose coug...
2,There is no evidence that garlic water vitamin...
3,11yearold Star Wars fan Isabella Tadlock was b...
4,Lettuce grown in space is not only tasty and s...
5,Heres how Danone yogurt went from being made i...
6,A quiet revolution is reshaping the agricultur...
7,On April 29 an asteroid estimated to be betwee...
8,Meet Rollerman Wearing a cyborgstyle suit made...
9,An Australian newspaper printed an extra eight...


Loading gensim and nltk libraries

In [20]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

C:\Users\marko\Anaconda3\envs\test_tf\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marko\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [37]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result
#answer = lemmatize_stemming('I was running when I went to the place siezing traditional agreed colonizer')
answer = lemmatize_stemming(no_http)
print(answer)

this caterpillar could potentially help solve one of the world's most pressing environmental problems: plastic wast… 


In [64]:
processed_tweets = df['Tweets'].map(preprocess)
processed_tweets[:10]

0    [polic, fulli, grow, footlong, american, allig...
1    [coronavirus, symptom, includ, runni, nose, co...
2    [evid, garlic, water, vitamin, essenti, oil, c...
3    [yearold, star, war, isabella, tadlock, bear, ...
4    [lettuc, grow, space, tasti, safe, astronaut, ...
5        [here, danon, yogurt, go, garag, take, world]
6    [quiet, revolut, reshap, agricultur, world, fa...
7    [april, asteroid, estim, mile, wide, earth, ex...
8    [meet, rollerman, wear, cyborgstyl, suit, carb...
9    [australian, newspap, print, extra, page, toil...
Name: Tweets, dtype: object

We now create a dictionary bag of words from the tweets

In [81]:
lexicon = gensim.corpora.Dictionary(processed_tweets)

count = 0
for k, v in lexicon.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
        
print(lexicon.token2id)

bow_corpus = [lexicon.doc2bow(doc) for doc in processed_tweets]
bow_corpus[86]

0 allig
1 american
2 basement
3 footlong
4 fulli
5 grow
6 home
7 homeown
8 live
9 ohio
10 polic
{'allig': 0, 'american': 1, 'basement': 2, 'footlong': 3, 'fulli': 4, 'grow': 5, 'home': 6, 'homeown': 7, 'live': 8, 'ohio': 9, 'polic': 10, 'coronavirus': 11, 'cough': 12, 'coupl': 13, 'fever': 14, 'headach': 15, 'includ': 16, 'nose': 17, 'runni': 18, 'sore': 19, 'symptom': 20, 'throat': 21, 'colloid': 22, 'essenti': 23, 'evid': 24, 'garlic': 25, 'oil': 26, 'protect': 27, 'silver': 28, 'steroid': 29, 'vitamin': 30, 'water': 31, 'bear': 32, 'finger': 33, 'isabella': 34, 'leav': 35, 'star': 36, 'tadlock': 37, 'war': 38, 'yearold': 39, 'astronaut': 40, 'diseasecaus': 41, 'free': 42, 'lettuc': 43, 'micro': 44, 'safe': 45, 'space': 46, 'tasti': 47, 'danon': 48, 'garag': 49, 'go': 50, 'here': 51, 'take': 52, 'world': 53, 'yogurt': 54, 'agricultur': 55, 'dani': 56, 'farmer': 57, 'like': 58, 'quiet': 59, 'reshap': 60, 'revolut': 61, 'say': 62, 'slabbert': 63, 'want': 64, 'april': 65, 'asteroid': 66

[(75, 1), (412, 1), (472, 1), (506, 1), (507, 1), (508, 1), (509, 1)]

In [82]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=lexicon, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.029*"coronavirus" + 0.027*"test" + 0.027*"offici" + 0.022*"posit" + 0.016*"local" + 0.016*"unit" + 0.012*"health" + 0.012*"case" + 0.011*"week" + 0.011*"announc"
Topic: 1 
Words: 0.029*"remain" + 0.019*"state" + 0.019*"unit" + 0.019*"outbreak" + 0.019*"prevent" + 0.019*"encourag" + 0.019*"center" + 0.019*"control" + 0.019*"diseas" + 0.019*"amid"
Topic: 2 
Words: 0.031*"resid" + 0.025*"coronavirus" + 0.020*"northern" + 0.020*"town" + 0.020*"hour" + 0.020*"italian" + 0.020*"realiz" + 0.020*"lambrusco" + 0.020*"castelvetro" + 0.018*"outbreak"
Topic: 3 
Words: 0.020*"woman" + 0.015*"race" + 0.015*"million" + 0.015*"biden" + 0.013*"quarantin" + 0.013*"worri" + 0.013*"aboard" + 0.013*"cancer" + 0.013*"california" + 0.013*"ship"
Topic: 4 
Words: 0.021*"coronavirus" + 0.011*"particular" + 0.011*"world" + 0.011*"moment" + 0.011*"video" + 0.011*"peopl" + 0.011*"spread" + 0.011*"grand" + 0.011*"cruis" + 0.011*"expect"
Topic: 5 
Words: 0.021*"govern" + 0.018*"amid" + 0.017*"prev